In [37]:
import os
import ee
import csv
import geemap
import geemap.colormaps as cm
import numpy as np
from glob import glob
import ipywidgets as widgets
from ipyleaflet import Map, TileLayer, DrawControl
from ipywidgets import Label
# Authenticate to Earth Engine
ee.Authenticate()
ee.Initialize()

In [24]:
def display_rgb_image_arrow(Map, image):
    Map.centerObject(image.geometry(), 8)
    Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')
    Map.addLayerControl()

def update_map(image_index):
    selected_image = ee.Image(image_list.get(int(image_index)))
    Map = display_rgb_image(selected_image)
    
    # Retrieve and print image ID
    image_id = selected_image.get('system:index').getInfo()
    print("Image ID:", image_id)
    
    return Map

def get_sentinel2_images(bbox, start_date, end_date, max_cloud_cover):
    region = ee.Geometry.Rectangle(bbox)
    date_filter = ee.Filter.date(start_date, end_date)
    cloud_filter = ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover)
    combined_filter = ee.Filter.And(date_filter, cloud_filter)

    sentinel2 = (ee.ImageCollection('COPERNICUS/S2')
                 .filterBounds(region)
                 .filter(combined_filter))

    return sentinel2

def get_sentinel2_image_by_id(s2):
    # Get the image based on its ID
    image_id = f"COPERNICUS/S2/{s2}"
    image = ee.Image(image_id)
    return image
def cloud_mask(image,b10thld):
    cld = image.select('B10')
    # Apply threshold to band 10 radiance
    cldmsk = cld.lte(b10thld)
    return cldmsk
def create_roi(csvpath):
    csv_data = []

    with open(fpath, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if not row[0][0] == 'L':
                lat, lon = float(row[1]), float(row[0])
                csv_data.append((lon, lat))
    if len(csv_data) > 0:
        # Create a polygon geometry from the CSV data
        roi = ee.Geometry.Polygon(csv_data)
        return roi
    else:
        return -10

In [13]:
def calculate_FAI(image):
    # Spectral reflectance values
    NIR_band = image.select('B8')  # Near-infrared band
    red_band = image.select('B4')  # Red band
    SWIR_band = image.select('B11')  # Shortwave infrared band
    
    # Wavelengths (in nm)
    NIR_lambda = 842  # S2 NIR band wavelength
    red_lambda = 665  # S2 red band wavelength
    SWIR_lambda = 1610  # S2 SWIR band wavelength
    
    wave_base = (NIR_lambda-red_lambda)/(SWIR_lambda-red_lambda)
    SWIR_red_diff = (SWIR_band.subtract(red_band)).multiply(wave_base)
    RP = red_band.add(SWIR_red_diff)
    # Compute FAI
    FAI = NIR_band.subtract(RP)
    return FAI

In [26]:
bbox = [-57,62,-51,65]
start_date = '2017-07-01'
end_date = '2017-08-01'
max_cloud_cover = 10

# Get Sentinel-2 images
sentinel2_images = get_sentinel2_images(bbox, start_date, end_date, max_cloud_cover)

# Convert image collection to a list for manual slider
image_list = sentinel2_images.toList(sentinel2_images.size())
total_images = sentinel2_images.size().getInfo()

# Initial index
current_index = 0

# Create a global Map object
Map = geemap.Map()
display(Map)

# Display the first image
selected_image = ee.Image(image_list.get(current_index))
display_rgb_image_arrow(Map, selected_image)
image_id = selected_image.get('system:index').getInfo()
print("Image ID:", image_id)

# Create left and right arrow buttons
button_layout = widgets.Layout(width='50px', height='30px')
left_arrow = widgets.Button(description='←', layout=button_layout)
right_arrow = widgets.Button(description='→', layout=button_layout)

# Function to handle left arrow click
def on_left_arrow_click(b):
    global current_index
    current_index = (current_index - 1) % total_images
    update_map()

# Function to handle right arrow click
def on_right_arrow_click(b):
    global current_index
    current_index = (current_index + 1) % total_images
    update_map()

# Function to update the map
def update_map():
    Map.remove('RGB')
    selected_image = ee.Image(image_list.get(current_index))
    display_rgb_image_arrow(Map, selected_image)
    image_id = selected_image.get('system:index').getInfo()
    print("Image ID:", image_id)
    
# Attach click event handlers to the buttons
left_arrow.on_click(on_left_arrow_click)
right_arrow.on_click(on_right_arrow_click)

# Display the buttons
display(widgets.HBox([left_arrow, right_arrow]))

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Image ID: 20170701T151909_20170701T152119_T22VDR


Image ID: 20170703T150911_20170703T151120_T22VEQ
Image ID: 20170705T145919_20170705T145936_T21VXL
Image ID: 20170705T145919_20170705T145936_T22VCR
Image ID: 20170705T145919_20170705T145936_T22VDP
Image ID: 20170705T145919_20170705T145936_T22VDQ
Image ID: 20170705T145919_20170705T145936_T22VDR
Image ID: 20170705T145919_20170705T145936_T22VEP
Image ID: 20170706T151911_20170706T152002_T22VDR
Image ID: 20170706T151911_20170706T152002_T22WDS
Image ID: 20170706T151911_20170706T152002_T22WDT
Image ID: 20170708T150909_20170708T150929_T22VEQ
Image ID: 20170710T145911_20170710T150120_T21WXM
Image ID: 20170710T145911_20170710T150120_T21WXN
Image ID: 20170710T145911_20170710T150120_T22VEQ
Image ID: 20170710T145911_20170710T150120_T22VER
Image ID: 20170710T145911_20170710T150120_T22WDS
Image ID: 20170710T145911_20170710T150120_T22WDT
Image ID: 20170710T145911_20170710T150120_T22WES


In [48]:
s2ids = ['20170710T145911_20170710T150120_T22WDS','20170710T145911_20170710T150120_T21WXM']
image = get_sentinel2_image_by_id(s2ids[1])
FAI = calculate_FAI(image)

In [49]:
import matplotlib.pyplot as plt
# Create a colormap for FAI visualization
cmap = plt.cm.get_cmap('jet')  # Choose a colormap

# plot RGB and FAI
Map = geemap.Map()

Map.centerObject(image.geometry(), 6)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 1000}, 'RGB')
pal = cm.palettes.ndwi

# Add the FAI layer to the map
# Add FAI image to the map
Map.addLayer(FAI, {'min': 0, 'max': 1, 'palette': ['blue', 'green', 'red']}, 'Floating Algae Index (FAI)')
# Add colorbar to the map
Map.add_colorbar(cmap=cmap, vmin=0, vmax=1, label='Floating Algae Index (FAI)')

Map.addLayerControl()

display(Map)

Map(center=[64.33387422395703, -53.214195889100274], controls=(WidgetControl(options=['position', 'transparent…